In [ ]:
import pandas as pd
import os
import re

In [ ]:
df = pd.read_csv('test_1.csv', header=None, names = ['topic','title', 'text'])

In [ ]:
df

In [ ]:
def tokenization(text):
    email_regex = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
    phone_regex = "\+?[1-9]\-?\(?[0-9]{3}\)?\-?[0-9]{2,4}\-?[0-9]{2,4}"
    link_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    word_regex = r'\w+'

    combined_regex = "|".join((email_regex, phone_regex, link_regex, word_regex))

    tokens = re.findall(combined_regex, text)

    return tokens

In [ ]:
tokenization("This fvfv +47854434555  had221@gmd.com callicarpa epimedium churchyard hangs; over the law the shadow. His callicarpa, part, may - not a; bit that all the. Anything - penstemon, had - forgotten you - may have now. Her, stokesia as if we meet van helsing i can have? Reading epimedium campanula; in time his stately; gravity; i presume, that you. Prevalent; campanula by pirates off on. I dined too prejudiced https://bit.ly/tfjqpibm you - oh, mina write))))))) Phrase betula berberis - callicarpa callicarpa; project gutenberg tm electronic works 1 e never. Panes stokesia berberis callicarpa there is to see that. And - rudbeckia https://bit.ly/oaghfwcj penstemon, monarda three of one - might frighten. ,flowers")

### Стэмминг и лемматизация nltk

In [ ]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

stemmer_en = SnowballStemmer('english')

lemmatizer = WordNetLemmatizer()

def convert_pos_wordnet(pos):
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def stemming(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer_en.stem(word) for word in words]
    return ' '.join(stemmed_words)

def lemmatization(text):
    words = nltk.word_tokenize(text)
    tagged_words = nltk.pos_tag(words)
    wordnet_tags = [convert_pos_wordnet(tag[1]) for tag in tagged_words]
    lemmatized_words = [lemmatizer.lemmatize(tag[0], tag[1]) for tag in zip(words, wordnet_tags)]
    return ' '.join(lemmatized_words)



In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

def split_into_sentences(text: str):


    text = text.replace("...","<three_points><stop>")
    text = text.replace(".",".<stop>")
    text = text.replace("??","<two_q><stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<three_points>","...")
    text = text.replace("<two_q>","??")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    return sentences[:-1]

def process_text(text):
    sentences = split_into_sentences(text)

    tokens_stems_lemmas = []
    for sentence in sentences:
        tokens = tokenization(sentence)
        stems = [stemmer_en.stem(token) for token in tokens]
        lemmas = [lemmatizer.lemmatize(token) for token in tokens]
        tokens_stems_lemmas.append(list(zip(tokens, stems, lemmas)))

    return tokens_stems_lemmas

df['processed_text'] = df['text'].apply(process_text)

In [ ]:
text = "bisfg fgfdg. sfg fgweon? fgdsf uijuijui hbh. iuiniuni jni ! YUUHiininifd sindf hinhisdf... dg iihifhihifhi."
split_into_sentences(text)

In [ ]:
df

In [ ]:
def create_annotation(df):
    for i in df['topic'].unique():
        path = 'assets/annotated_corpus/test/'+str(i)
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)
        
    for index, row in df.iterrows():
        output_filename = f"assets/annotated_corpus/test/{str(row['topic'])}/annotation_{index}.tsv"
        with open(output_filename, "w", encoding="utf-8") as f:
            for sentence in row['processed_text']:
                for token, stem, lemma in sentence:
                    f.write(f"{token}\t{stem}\t{lemma}\n")
                f.write("\n")  


create_annotation(df)